# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** This notebook acts as an index to all the other notebooks

## Table of contents <a name="table-of-contents"></a>

1. [Table of notebooks](#table-notebooks)
2. [Introduction](#introduction)
3. [Collecting the data](#collecting-the-data)

## 1. Table of notebooks <a name="table-notebooks"></a>

1. [Index notebook (current notebook)](./1_entry_notebook.ipynb)
2. [Getting the data](./2_get_data.ipynb)
3. [Some Exploratory Data Analysis (EDA)](./3_exploratory_data_analysis.ipynb)
4. [Selecting our model](./4_model_selection.ipynb)
5. [Fitting the data and predicting the future!](./5_fitting_and_predicting.ipynb)

## 2. Introduction <a name="introduction"></a>

This notebook will give a brief overview of what the assignment was about, how we handled it and a small conclusion.

The goal of the assignment was to predict the temperature values one hour and one day ahead of time. This was done by first collecting temperature data from a chosen location with the [CC3200 LaunchPad](http://www.ti.com/tool/CC3200-LAUNCHXL).


The data the LaunchPad collected was stored in a database hosted on [heroku](https://heroku.com). An API to upload the data was also hosted on there.

Once enough data is collected it was possible to build these notebooks to accually predict the temperature. This was done by using the [Python](https://www.python.org/) Programming Language in combination with some [modules](./requirements.txt) and [anaconda](https://www.anaconda.com/).

Using Seasonal ARIMAX the future is predicted.

## 3. Collecting the data <a name="collecting-the-data"></a>

